<a href="https://colab.research.google.com/github/Mohitg06/pneumoscan-Lab-PPE/blob/master/Pneunmoscan_Lab_PPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initializing a model using AWS SageMaker Python API

In [0]:
import json 
from sagemaker import ModelPackage
from src.model_package_arns import ModelPackageArnProvider
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
sess = sage.Session()
role = get_execution_role()
region=sess.boto_region_name

In [0]:
from utils import get_model_package_arn

model_package_arn = get_model_package_arn(sess.boto_region_name)

model = sage.ModelPackage(
    role=role,
    model_package_arn=model_package_arn)

2. Generating Prediction using RealTimePredictor
Creating a new endpoint

In [0]:
endpoint_name='ppe-lab-model-endpoint'

model.deploy(initial_instance_count=1, instance_type='ml.m4.2xlarge', endpoint_name=endpoint_name)

In [0]:
#Define a generic image predictor wrapper which accepts endpoint & session object, and returns a predictor wrapper
def image_predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type='image/jpeg')

#### Generating predictions

In [0]:
predictor = sage.predictor.RealTimePredictor(
    endpoint_name,
    sagemaker_session=sess, 
    content_type="image/jpeg"
)

In [0]:
file_name = 'sample_data/input/image2.jpg'

with open(file_name, 'rb') as image:
    f = image.read()
    image_bytes = bytearray(f)

prediction_result = predictor.predict(image_bytes).decode("utf-8")

In [0]:
import json

result = json.loads(prediction_result)
result

Analyze Video

In [0]:
from IPython.display import HTML
video_path='./video/video.mp4'

HTML('<iframe width="560" height="315" src="'+video_path+'?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

In [0]:
capture = cv2.VideoCapture(video_path) 

#Get number of frames from the video.
framecount = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

#Take snapshot every 3 second(s)
num_seconds = 3

skip_frames=capture.get(cv2.CAP_PROP_FPS)*num_seconds

num_snapshots=int(framecount/skip_frames)

#bucket = sess.boto_session.resource('s3').Bucket(sess.default_bucket())

#For this experiment, we extract an image every second so that we can utilize the endpoints more efficiently.
for i in range(num_snapshots):
    flag, frame = capture.read()
    if flag:
        path = './video/frames/frame' + str(i) + '.jpg'
        print ('Creating snapshot on path - ' + path) 
        cv2.imwrite(path, frame) 
        capture.set(cv2.CAP_PROP_POS_FRAMES, ((i+1)*skip_frames))
capture.release()

In [0]:
def generate_status_summary(image_path):
    image_byte_array = []

    num_cap = 0
    num_ppe = 0
    num_face_mask = 0
    num_gown=0

    # Open the image.
    with open(image_path, "rb") as image:
        image_byte_array = bytearray(image.read())

    # Count number of gowns and caps
    machine_detection_result = json.loads(predictor.predict(image_byte_array).decode('utf-8'))
    for output in machine_detection_result['outputs']['detections']:
        if output[5] > 0.65:
            if output[4] == 10:
                num_cap += 1
            if output[4] == 15:
                num_gown += 1

    # Count number of personal protective equipments(PPEs)
    ppe_detection_result = json.loads(predictor.predict(image_byte_array).decode('utf-8'))
    for output in ppe_detection_result['output']:
        if output['score'] > 0.5:
            num_ppe += 1

    # Count number of face masks
    face_mask_detection_result = json.loads(predictor_hard_hat_detection.predict(image_byte_array).decode('utf-8'))
    for i in range(len(face_mask_detection_result['boxes'])):
        if face_mask_detection_result['scores'][i] > 0.5:
            num_face_mask += 1

    # Create and return the summary.
    if num_ppe == num_face_mask == 0:
        current_status = "Alarm : " + str(num_gown) + " gown(s), " + str(
            num_cap) + " cap(s), no medical staff found."
    elif (num_ppe == num_face_mask):
        current_status = "No Alarm : " + str(num_gown) + " gown(s), " + str(num_cap) + " cap(s), " + str(num_face_mask)
        "face mask(s)" + str( num_ppe) + " medical staff found."
    elif num_ppe > num_face_mask:
        current_status = "ALARM    : " + str(num_ppe) + " Medical Workers(s) wearing PPE but " + str(
            num_face_mask) + " wearing face masks, " + str(num_gown) + " gown(s), " + str(
            num_cap) + " cap(s) found."
    elif num_face_mask > num_ppe:
        current_status = "ALARM    : " + str(num_face_mask) + " Medical Worker(s) wearing face masks but " + str(
            num_ppe) + " workers wearing PPE, " + str(num_gown) + "gown(s), and " + str(
            num_cap) + " cap(s) found."
    return current_status